<a href="https://colab.research.google.com/github/Chhaya-cloud/Encoder-Decoder-Architecture/blob/main/Encoder_Decoder_Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Please connect to v5e-1 TPU runtime.

In [ ]:
# Installing required libraries
!pip install tensorflow tensorflow_datasets

In [ ]:
# Importing important libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow_datasets as tfds
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Loading dataset from Tensorflow Datasets
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True, as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/incomplete.6YNZ2Y_1.0.0/ted_hrlr_translate-val…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/incomplete.6YNZ2Y_1.0.0/ted_hrlr_translate-tes…

Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [ ]:
# We'll only use a small subset for demonstration
MAX_SAMPLES = 1000

In [ ]:
# Function for preprocessing the data
def preprocess(data, num_samples = MAX_SAMPLES):
  src_texts = []
  tgt_texts = []

  for src, tgt in data.take(num_samples):
    src = src.numpy().decode('utf-8')
    tgt = tgt.numpy().decode('utf-8')
    tgt = '<start> ' + tgt + ' <end>'
    src_texts.append(src)
    tgt_texts.append(tgt)

  return src_texts, tgt_texts

In [ ]:
# Driver Code
input_texts, target_texts = preprocess(train_examples)

In [ ]:
# Tokenization
def tokenize(sentences):
  tokenizer = Tokenizer(filters = '')
  tokenizer.fit_on_texts(sentences)
  tensor =  tokenizer.texts_to_sequences(sentences)
  tensor = pad_sequences(tensor, padding = 'post')
  return tensor, tokenizer

In [ ]:
# Driver Code
input_tensor, input_tokenizer = tokenize(input_texts)
target_tensor, target_tokenizer = tokenize(target_texts)

In [ ]:
# Vocabulary Sizes
input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

In [ ]:
# Homework -> Research on why we do '+1'

In [ ]:
# Sequences lengths
max_input_len = input_tensor.shape[1]
max_target_len = target_tensor.shape[1]

# Encoder - Decoder with LSTM
embedding_dim = 256
units = 512

In [ ]:
# Encoder
encoder_inputs = Input(shape = (None, ))
enc_emb = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(units, return_state = True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [ ]:
# Decoder
decoder_inputs = Input(shape = (None, ))
dec_emb = Embedding(target_vocab_size, embedding_dim)(decoder_inputs)
dec_lstm = LSTM(units, return_sequences = True, return_state = True)
decoder_outputs, _, _ = dec_lstm(dec_emb, initial_state = encoder_states)
decoder_dense = Dense(target_vocab_size, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# Compile the model
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy')

In [ ]:
# Prepare decoder target data by shifting
target_input = target_tensor[:, :-1]
target_output = target_tensor[:, 1:]
target_output = np.expand_dims(target_output, axis = -1) # Keras needs 3D target

In [ ]:
# Train the model
model.fit([input_tensor, target_input],
          target_output, batch_size = 32, epochs = 3)

Epoch 1/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 168s 5s/step - loss: 3.4507
Epoch 2/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 167s 5s/step - loss: 0.8594
Epoch 3/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 165s 5s/step - loss: 0.8165


In [ ]:
# Model Summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 256) │  1,006,848 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 256) │    848,128 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 512),     │  1,574,912 │ embedding[0][0]   │
│                     │ (None, 512),      │            │                   │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │  1,574,912 │ embedding_1[0][0… │
│                     │ 512), (None,      │            │ lstm[0][1],       │
│                     │ 512), (None,      │            │ lstm[0][2]        │
│                     │ 512)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None,      │  1,699,569 │ lstm_1[0][0]      │
│                     │ 3313)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 20,113,109 (76.73 MB)

 Trainable params: 6,704,369 (25.58 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 13,408,740 (51.15 MB)

In [ ]:
# Research on "bleu_score" and how it is used to evaluate text-to-text conversion models.